In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("SQ.csv")
df = df.drop(columns=["Date"])
print (df)

In [ ]:
X = df.to_numpy()
print (X)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
price = df[["Close"]]
scaler = MinMaxScaler(feature_range=(-1,1))
Y = scaler.fit_transform(price["Close"].values.reshape(-1,1))
X = scaler.fit_transform(X)

In [ ]:
print(X)
#print(Y)

In [ ]:
def split_data(X, Y, lookback, split_ratio):
    
    data = []

    for index in range(len(X) - lookback): 
        data.append(X[index: index + lookback])

    data = np.array(data);
    test_set_size = int(np.round(split_ratio * data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    x_train = data[:train_set_size,:]
    y_train = Y[:train_set_size,:]
    
    x_test = data[train_set_size:,:]
    y_test = Y[train_set_size:,:]
    
    return [x_train, y_train, x_test, y_test]

lookback = 40 # sequence length
split_ratio = 0.1
x_train, y_train, x_test, y_test = split_data(X, Y, lookback, split_ratio)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


In [ ]:
import torch
import torch.nn as nn

x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

input_dim = 6
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

learning_rate = 0.01

class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(MyModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = MyModel(input_dim = input_dim, hidden_dim = hidden_dim, num_layers = num_layers, output_dim = output_dim)
loss_func = torch.nn.MSELoss(reduction='mean')
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
import time
n_epochs = 100
start_time=time.time()
for e in range(n_epochs):
  y_train_pred = model(x_train)
  loss = loss_func(y_train_pred, y_train)
  print(f"Epoch: {e}\tLoss: {loss.item()}")
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
elapsed_time = time.time() - start_time
print(f"Total time for training: {elapsed_time}")

In [ ]:
# how do you use this on current data in order to get a predicted number?

# feed current data into this model?

In [ ]:
## Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
 
api = tradeapi.REST(
   alpaca_api_key,
   alpaca_secret_key,
   api_version = "v2"
)
# Set timeframe to '1D'
timeframe = "1D"
# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()
# Set the ticker information
tickers = ["MSFT","KO"]
# Get 3 year's worth of historical price data for Microsoft and Coca-Cola
df_ticker = api.get_barset(
   tickers,
   timeframe,
   start=start_date,
   end=end_date,
   limit=1000,
).df
# Display sample data
df_ticker.head()